In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


This notebook contains the code used to create the vector store, as well as do some preliminary testing for the final RAG class that I built.


In [ ]:
#openai_api_key
api_key = "YOUR OPENAI KEY"

In [ ]:
!pip install langchain
!pip install openai
!pip install faiss-cpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
import tiktoken

In [ ]:
llm_model = "gpt-3.5-turbo"

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
data = TextLoader('/content/drive/MyDrive/Arunadiss/main/scrape_bioninja/ib_bio_hl.txt', encoding="utf-8").load()


text_splitter = CharacterTextSplitter(
    separator="###",
    chunk_size=500,
    chunk_overlap=0,
    is_separator_regex=False,
)
data = text_splitter.split_documents(data)


In [ ]:
len(data)


693

In [ ]:
!pip install -U langchain-openai

In [ ]:
from langchain_openai import OpenAIEmbeddings
# Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [ ]:
folder_path = "/content/drive/MyDrive/Arunadiss/main/RAG/v0/vector_store"

# Save the vector store to disk
vectorstore.save_local(folder_path, index_name="vectorstore")


In [ ]:
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS




In [ ]:
query = "What is variation"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

5.2 Natural Selection
Variation:
Understanding:•  Natural selection can only occur if there is variation among members of the same species•  Mutation, meiosis and sexual reproduction causes variation between individuals in a species
Natural selection can only occur if there is variation among members of the same species•  Mutation, meiosis and sexual reproduction causes variation between individuals in a species Natural selection requires variation among members of a species in order to differentiate survival (variation needed for selection) This variation can manifest as either discontinous (distinct classes) or continuous (range across a characteristic spectrum) There are three main mechanisms by which genetic variation between individuals in a species may occur: Mutations – Changing the genetic composition of gametes (germline mutation) leads to changed characteristics in offspring Meiosis – Via either crossing over (prophase I) or independent assortment (metaphase I) Sexual reprodu

In [ ]:
query = "What is variation"
docs = vectorstore.similarity_search(query,k=2)
print(len(docs))


2


In [ ]:
def get_context(question):
    docs=vectorstore.similarity_search(question,k=3)
    list_of_docs=[]
    for doc in docs:
      list_of_docs.append(doc.page_content)
    return list_of_docs

In [ ]:
def generate_response(question,content_prompt,model, rag):

    context= get_context(question) if rag ==True else ""
    context_str=''
    for i in context:
        context_str+=i+'\n\n'

    prompt = (

        f"Question: {question} \n\n"
        f"Context: {context}"
        "Respond using context."
    )


    # Use OpenAI API to generate the email response
    response = client.chat.completions.create(
        model= model,#"gpt-4", #gpt-3.5-turbo
        messages=[
            {
                "role": "system",
                "content": (content_prompt)
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=0.7,
        max_tokens=500,
        top_p=1
    )
    tokens_input = response.usage.prompt_tokens
    tokens_output = response.usage.completion_tokens
    complete_response = response.choices[0].message.content

    # Return the generated response
    return [complete_response,context,(tokens_input,tokens_output)]


In [ ]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

### RAG

In [ ]:
query = "What is DNA Replication"
answer=generate_response(query,"You are an AI assistant","gpt-4",True)

print(answer[0])
print('\n\n')
print(answer[1])
print('\n\n')
print(answer[2])

DNA replication is a semi-conservative process carried out by a complex system of enzymes, where an existing DNA molecule serves as a template for creating a new DNA molecule. This process starts from specific sequences called origins of replication. 

Enzymes involved in DNA replication include:

1. Helicase: It unwinds and separates the double-stranded DNA by breaking the hydrogen bonds between base pairs, creating a replication fork of two strands running in antiparallel directions.

2. DNA Gyrase: It reduces the torsional strain created by the unwinding of DNA by helicase, relaxing positive supercoils that would otherwise form during the unwinding.

3. Single Stranded Binding (SSB) Proteins: These bind to the DNA strands after they have been separated, preventing the strands from re-annealing and from being digested by nucleases.

4. DNA Primase: This generates a short RNA primer on each of the template strands, providing an initiation point for DNA polymerase III.

5. DNA Polymera

In [ ]:
query = "What is DNA Replication"
answer=generate_response(query,"You are an AI assistant","gpt-3.5-turbo",True)

print(answer[0])
print('\n\n')
print(answer[1])
print('\n\n')
print(answer[2])

DNA replication is the process by which a cell makes an identical copy of its DNA. It is a crucial biological process that ensures genetic information is accurately passed on to daughter cells during cell division. In DNA replication, the double-stranded DNA molecule is unwound by an enzyme called helicase, which separates the two strands by breaking the hydrogen bonds between base pairs. This creates a replication fork with two strands running in opposite directions.

Several enzymes are involved in DNA replication, including DNA gyrase, single-stranded binding (SSB) proteins, DNA primase, DNA polymerase III, DNA polymerase I, and DNA ligase. These enzymes work together to ensure the accurate and efficient replication of the DNA molecule.

DNA polymerase III plays a central role in DNA replication by synthesizing new DNA strands from the parental template strands. It attaches to the 3'-end of a primer and covalently joins free nucleotides together in a 5' → 3' direction. DNA polymeras

In [ ]:

content_prompt = """
You are an AI assistant designed to help learners using the INSPIRE framework, which emphasizes Intelligent, Nurturant, Socratic, Progressive, Indirect, Reflective, and Encouraging approaches in tutoring.

- As an Intelligent tutor, use your vast knowledge base to provide detailed explanations, incorporate relevant historical context, employ visual aids, and draw real-world analogies to clarify concepts.

- Be Nurturant by showing empathy towards learners' struggles, offering supportive feedback, and encouraging perseverance and resilience.

- Adopt a Socratic approach by asking open-ended questions that encourage critical thinking and exploration, rather than providing direct answers.

- Ensure a Progressive learning experience by adjusting the complexity of questions and topics based on the learner's progress and responses, gradually challenging them with more difficult material.

- Provide Indirect guidance by offering hints and nudges towards solutions instead of outright answers, encouraging learners to think independently and develop problem-solving skills.

- Foster Reflective thinking by prompting learners to verbalize their thought processes, reflect on their answers, and learn from both their successes and mistakes.

- Be Encouraging by highlighting learners' achievements, motivating them to tackle challenging problems, and maintaining a positive, empowering learning environment.

Remember, you will be provided with context for each interaction, allowing you to tailor your approach to meet the specific needs and understanding level of each learner. Your goal is to make the learning experience as engaging, effective, and supportive as possible.
"""

query = "What is DNA Replication"

answer=generate_response(query,content_prompt,"gpt-4",True)

print(answer[0])
print('\n\n')
print(answer[1])
print('\n\n')
print(answer[2])



DNA replication is a semi-conservative process that involves the creation of an identical copy of DNA from a DNA molecule. This is a crucial process as it allows the genetic information to be passed down from cell to cell during cell division. 

The process begins at specific sequences called origins of replication. The DNA molecule is split into two strands by an enzyme called helicase, which breaks the hydrogen bonds between the base pairs. DNA gyrase helps reduce the strain created by the unwinding of the DNA. As these strands separate, single-strand binding proteins bind to the strands to prevent them from re-annealing and protect them from being digested by nucleases.

To initiate the process of adding new nucleotides, DNA primase generates a short RNA primer on each of the template strands. DNA polymerase III then attaches to the 3’-end of the primer and begins adding nucleotides in a 5’ to 3’ direction, complementary to the template strand. 

Because DNA strands are antiparallel

In [ ]:
query = "What is DNA Replication"

answer=generate_response(query,content_prompt,"gpt-3.5-turbo",True)

print(answer[0])
print('\n\n')
print(answer[1])
print('\n\n')
print(answer[2])

DNA replication is a fundamental process where a cell duplicates its DNA before cell division. It ensures that each daughter cell receives an identical copy of the genetic material. In the context provided, DNA replication involves several key enzymes and steps that work together in a highly orchestrated manner.

One crucial enzyme involved in DNA replication is helicase. Helicase plays a vital role in unwinding the double-stranded DNA molecule by breaking the hydrogen bonds between the base pairs. This unwinding process creates what is known as a replication fork, where the DNA strands separate, exposing the template strands for replication.

Another essential enzyme is DNA polymerase, which is responsible for linking nucleotides together to form a new DNA strand. During replication, DNA polymerase uses the pre-existing DNA strand as a template to guide the synthesis of a complementary strand. This process ensures the accurate replication of genetic information.

Additionally, DNA rep

### No RAG

In [ ]:
query = "What is DNA Replication"
answer=generate_response(query,"You are an AI assistant","gpt-4",False)

print(answer[0])
print('\n\n')
print(answer[1])
print('\n\n')
print(answer[2])

As an AI, I don't have context provided, but in general, DNA replication is a biological process by which the two strands of the DNA molecule separate and each strand serves as a template to produce a complementary strand. This process results in the creation of two identical DNA molecules from a single DNA molecule and is crucial for cell division and growth. The sequence of the DNA is copied by a complex set of enzymes, including DNA polymerase. These enzymes ensure that each new cell has the same genetic information as the original cell.







(30, 105)


In [ ]:
query = "What is DNA Replication"
answer=generate_response(query,"You are an AI assistant","gpt-3.5-turbo",False)

print(answer[0])
print('\n\n')
print(answer[1])
print('\n\n')
print(answer[2])

DNA replication is the biological process by which a cell makes an identical copy of its DNA. This is a crucial step in cell division, as it ensures that each new cell receives a complete set of genetic information. The process involves separating the DNA strands, synthesizing new complementary strands using the existing strands as templates, and proofreading to ensure accuracy. DNA replication is essential for growth, development, and reproduction in all living organisms.







(30, 85)


In [ ]:

content_prompt = """
You are an AI assistant designed to help learners using the INSPIRE framework, which emphasizes Intelligent, Nurturant, Socratic, Progressive, Indirect, Reflective, and Encouraging approaches in tutoring.

- As an Intelligent tutor, use your vast knowledge base to provide detailed explanations, incorporate relevant historical context, employ visual aids, and draw real-world analogies to clarify concepts.

- Be Nurturant by showing empathy towards learners' struggles, offering supportive feedback, and encouraging perseverance and resilience.

- Adopt a Socratic approach by asking open-ended questions that encourage critical thinking and exploration, rather than providing direct answers.

- Ensure a Progressive learning experience by adjusting the complexity of questions and topics based on the learner's progress and responses, gradually challenging them with more difficult material.

- Provide Indirect guidance by offering hints and nudges towards solutions instead of outright answers, encouraging learners to think independently and develop problem-solving skills.

- Foster Reflective thinking by prompting learners to verbalize their thought processes, reflect on their answers, and learn from both their successes and mistakes.

- Be Encouraging by highlighting learners' achievements, motivating them to tackle challenging problems, and maintaining a positive, empowering learning environment.

Remember, you will be provided with context for each interaction, allowing you to tailor your approach to meet the specific needs and understanding level of each learner. Your goal is to make the learning experience as engaging, effective, and supportive as possible.
"""

query = "What is DNA Replication"

answer=generate_response(query,content_prompt,"gpt-4",False)

print(answer[0])
print('\n\n')
print(answer[1])
print('\n\n')
print(answer[2])


DNA replication is a fundamental process that occurs in all living organisms to copy their DNA. This process is vital because, whenever a cell divides, the two new daughter cells must contain the same genetic information, or DNA, as the parent cell. 

Imagine a long ladder that's twisted into a spiral. This is the structure of DNA, a double helix. Now, imagine this ladder splitting down the center, with each rung of the ladder breaking apart. Each half can serve as a template for a new DNA molecule. 

Now, let's dive a bit deeper. During DNA replication, enzymes called helicases unwind the DNA double helix. Then, another enzyme, DNA polymerase, adds nucleotides (the building blocks of DNA: adenine, thymine, cytosine, guanine) to each template strand. 

The DNA polymerase reads the original strand and adds the corresponding nucleotide. For example, if it encounters an adenine on the template strand, it will add a thymine to the new strand. This is because adenine and thymine, like cytos

In [ ]:
query = "What is DNA Replication"

answer=generate_response(query,content_prompt,"gpt-3.5-turbo",False)

print(answer[0])
print('\n\n')
print(answer[1])
print('\n\n')
print(answer[2])

DNA replication is a fundamental process in biology where a cell makes an identical copy of its DNA. This process is crucial for cell division, growth, and repair. During DNA replication, the double-stranded DNA molecule unwinds and separates into two strands. Each strand serves as a template for the creation of a new complementary strand, resulting in two identical DNA molecules.

To help you visualize this process, think of DNA as a zipper that needs to be unzipped for replication to occur. The enzyme helicase acts like the zipper pull, unzipping the DNA strands. Other enzymes, such as DNA polymerase, then work to build new strands by adding complementary nucleotides to each template strand. This results in two complete and identical DNA molecules.

Understanding DNA replication is key to comprehending how genetic information is passed on from one generation to the next and how mutations can occur. If you have any specific questions or need further clarification, feel free to ask!


